## 2D Data modelling
I am going to work on a ml model to predict prices using variable2

In [1]:
# Importing our varibales
import matplotlib.pyplot as plt
import pandas as pd
# Machine learning: regression model + metrics
from sklearn.linear_model import LinearRegression  
from sklearn.metrics import mean_absolute_error
from sklearn.utils.validation import check_X_y, check_is_fitted 

print("All requirements satisfied!")

All requirements satisfied!


## Preparing our data
Importing

In [ ]:
#